In [5]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from scipy.stats import randint
from pprint import pprint
from sklearn.tree import export_graphviz
from IPython.display import Image
#import graphviz
from sklearn import decomposition
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D

In [4]:
fraud = pd.read_csv('../data/fraud_condensed_new.csv')

In [8]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)

In [ ]:
y = fraud['isFraud']
x = fraud.drop('isFraud', axis = 1)

# Train, test splits with test size containing 30% of the original data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [4]:
#scale = StandardScaler()
#scale.fit(x_train)
#scale.transform(x_train)

array([[ 0.48068052,  0.38015847, -0.20791103, ..., -0.02075243,
         0.        ,  0.        ],
       [ 0.94881307, -0.65370991, -0.21119183, ..., -0.02075243,
         0.        ,  0.        ],
       [ 0.3116033 ,  0.40995932, -0.44140014, ..., -0.02075243,
         0.        ,  0.        ],
       ...,
       [-0.40486143,  1.59339275, -0.41822042, ..., -0.02075243,
         0.        ,  0.        ],
       [-0.86348339, -1.11655951,  0.51025038, ..., -0.02075243,
         0.        ,  0.        ],
       [ 1.58179592,  0.27191689, -0.41395089, ..., -0.02075243,
         0.        ,  0.        ]])

In [4]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)

In [5]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9979919678714859


In [6]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.fit(x_train,y_train)

GradientBoostingClassifier()

In [7]:
print(classification_report(y_train,gradient_booster.predict(x_train)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2322
           1       1.00      1.00      1.00         1

    accuracy                           1.00      2323
   macro avg       1.00      1.00      1.00      2323
weighted avg       1.00      1.00      1.00      2323



In [8]:
for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=x_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

NameError: name 'graphviz' is not defined